In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from astropy import units
from pathlib import Path
import os

import snapshot_obj
import dataset_compute
import curve_fit

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)
importlib.reload(curve_fit)

# Differences between values of $v_\mathrm{max}$ 

To validate the values computed with my implemented methods, I compare them to the catalogued values that should be alright.

## Inspect subhalo sizes

First let us plot a histogram (logarithmically) of the number of bound particles of the subhalos in the med-res run. As expected, most halos are formed in the small-mass range, with only about $<100$ bound particles. These halos are hardly resolved.

In [ ]:
# Get LCDM data:
LCDM = snapshot_obj.Snapshot("V1_MR_fix", 127)
vmax = LCDM.get_subhalos("Vmax") * units.cm.to(units.km)
data = LCDM.get_subhalos("Max_Vcirc")
max_vcirc = data[:,0] * units.cm.to(units.km)
rmax = data[:,1] * units.cm.to(units.kpc)

In [ ]:
part_nums = LCDM.get_subhalos("SubLength")
print(np.min(part_nums))

In [ ]:
down = 1; up = 5
part_nums = np.log10(part_nums)
part_nums = part_nums[np.logical_and(down < part_nums, part_nums < up)]
fig, axes = plt.subplots()
axes.set_xlabel('$\mathrm{log}(N_\mathrm{bound})$')
axes.hist(part_nums, 10)
plt.show()

## Plot differing values

Now, let us find the subhalos, for which the difference in the $v_\textrm{max}$ value is in a given range. We see that subhalos with a greater disagreement are the subhalos with less particles. The lines that form in the small-mass range of the figure correspond to $n_\mathrm{soft}, n_\mathrm{soft}*2, n_\mathrm{soft}*3, \dots$ dm particles. The value of $n_\mathrm{soft}$ only makes a difference very close to the halo centre (where the accumulated number of particles is small). However, to have a meaningful value of $v_\mathrm{max}$ for the subhalos with around 50 particles, I chose $n_\mathrm{soft}=5$ (instead of 10 used for rotation curves). We certainly should not be concerned with the subhalos with less than 50 particles (and talking about halos this small is already pushing it a bit).

Looking at subhalos with 5% difference in the values, all of them are among the subhalos that are not really resolved yet. Being able to compute $v_\mathrm{max}$ for the interesting halos with 5% accuracy, in the very least, is sufficient-

In [ ]:
err = np.abs(vmax-max_vcirc)/vmax
differing = np.logical_and(0.01 < err, err < 1)
print(vmax.size)
print(np.sum(differing))

In [ ]:
fig, axes = plt.subplots()

# Set axis:
axes.set_xscale('log')
axes.set_yscale('log')
axes.set_xlim(0.1, 50)
axes.set_ylim(0.3, 100)

# Add LCDM scatter plot:
axes.scatter(rmax[differing], vmax[differing], s=3, label='catalogue')
axes.scatter(rmax[differing], max_vcirc[differing], s=3, label='me')

plt.legend()
plt.tight_layout()


# Check locations on rotation curves

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)

import snapshot
import simulation
import dataset_comp

In [ ]:
import importlib
importlib.reload(snapshot)
importlib.reload(dataset_comp)
importlib.reload(simulation)

## Construct data dictionary

Add entries for each simulation, and specify M31 and MW galaxies:

In [ ]:
snap_id = 127
sim_ids = ['V1_LR_fix', 'V1_LR_curvaton_p082_fix', 'V1_LR_curvaton_p084_fix']
names = ['LCDM', 'p082', 'p084']

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0), (1,0)]
mw = [(2,0), (1,1), (1,0)]

data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {'snapshot': snapshot.Snapshot(sim_id, snap_id, name=name),
                  'M31_identifier': m31_ns,
                  'MW_identifier': mw_ns}

In [ ]:
snap_id = 127
sim_ids = ['V1_LR_fix', 'V1_LR_curvaton_p082_fix']
names = ['LCDM', 'p082']

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0)]
mw = [(2,0), (1,1)]

data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {'snapshot': snapshot.Snapshot(sim_id, snap_id, name=name),
                  'M31_identifier': m31_ns,
                  'MW_identifier': mw_ns}

In [ ]:
snap_id = 127
sim_ids = ['V1_MR_fix', 'V1_MR_curvaton_p082_fix']
names = ['LCDM', 'p082']

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0)]
mw = [(2,0), (1,1)]

data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {'snapshot': snapshot.Snapshot(sim_id, snap_id, name=name),
                  'M31_identifier': m31_ns,
                  'MW_identifier': mw_ns}

Choose how to distinguish between satellite and isolated galaxies:

In [ ]:
distinction = 'by_r'
maxdi = 4000

In [ ]:
dx = 0.23
binc = np.array([(1.0+dx/2) + dx*n for n in range(4)])
dex_bins = np.array([[c-dx/2,c,c+dx/2] for c in binc])
sat_bins = 10**dex_bins
print(sat_bins)

In [ ]:
dx = 0.2
binc = np.array([(1.2+dx/2) + dx*n for n in range(4)])
dex_bins = np.array([[c-dx/2,c,c+dx/2] for c in binc])
isol_bins = 10**dex_bins
print(isol_bins)

Read datasets, split into satellites and isolated galaxies, and add to the data dictionary. We also disregard dark halos and potential spurious halos with $v_\mathrm{max} = 0$.

In [ ]:
for name, sim_data in data.items():
    # Get data:
    snap = sim_data['snapshot']
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    rmax = max_point[:,1] * units.cm.to(units.kpc)
    
    # Retrieve rotation curves and split by subhalo:
    rot_curves = snap.get_subhalos(
        'Vcirc', group='Extended/RotationCurve/All')
    sub_offset = snap.get_subhalos(
        'SubOffset', group='Extended/RotationCurve/All')
    v_circ = rot_curves[:,0] * units.cm.to(units.km)
    radii = rot_curves[:,1] * units.cm.to(units.kpc)
    v_circ = np.array(np.split(v_circ, sub_offset[1:]))
    radii = np.array(np.split(radii, sub_offset[1:]))
    
    # Split into satellites:
    if distinction == 'by_r':
        masks_sat, mask_isol = dataset_comp.split_satellites_by_distance(
            snap, sim_data['M31_identifier'], sim_data['MW_identifier'], max_dist_isol=maxdi)
    elif distinction == 'by_gn':
        masks_sat, mask_isol = dataset_comp.split_satellites_by_group_number(
            snap, sim_data['M31_identifier'], sim_data['MW_identifier'])
    
    # Compute masks for vmax bins:
    sat_bin_masks = [dataset_comp.prune_vmax(snap, low_lim=b[0], up_lim=b[2])
                     for b in sat_bins]
    isol_bin_masks = [dataset_comp.prune_vmax(snap, low_lim=b[0], up_lim=b[2])
                     for b in isol_bins]

    # Compute other masking arrays:
    mask_lum, mask_dark = dataset_comp.split_luminous(snap)
    
    # Add selections (masking arrays):
    data[name]['Selections'] = {
        'M31': masks_sat[0],
        'MW': masks_sat[1],
        'Satellite': np.logical_or.reduce(masks_sat),
        'Isolated': mask_isol,
        'VmaxBins': {
            'Satellite': {
                'Low': sat_bin_masks[0], 
                'SemiLow': sat_bin_masks[1],
                'SemiHigh': sat_bin_masks[2],
                'High': sat_bin_masks[3]  
            },
            'Isolated': {
                'Low': isol_bin_masks[0], 
                'SemiLow': isol_bin_masks[1],
                'SemiHigh': isol_bin_masks[2],
                'High': isol_bin_masks[3]  
            }
        },
        'Luminous': mask_lum,
        'Dark': mask_dark
    }
    
    # Add datasets to dictionary:
    data[name]['Vmax'] = vmax
    data[name]['Rmax'] = rmax
    data[name]['Vcirc'] = v_circ
    data[name]['Radius'] = radii

## Plot

In [ ]:
# Set some parameters:
x_down = 10; x_up = 100
y_down = 5*10**5; y_up = 2*10**10

# Set marker styles:
fcolor = ['black', 'red', 'blue', 'green']
mcolor = ['gray', 'pink', 'lightblue', 'lightgreen']
marker = ['+', 'o', '^', 1]
a=0.5

In [ ]:
# Construct saving location:
filename = 'rotation_curve_bundles_sat'
for name in names:
    filename += '_{}'.format(name)
filename += '.png'
    
home = os.path.dirname(snapshot.__file__)
path = os.path.join(home,'Figures', 'MediumResolution')
filename = os.path.join(path, filename)

In [ ]:
"$\log_{{10}}\\frac{{v_\mathrm{{max}}}}{{\mathrm{{km/s}}}} = {:.3f}".format(
               dex_bins[0,1])

In [ ]:
"${:.2f} \mathrm{{km/s}} < v_\mathrm{{max}} < {:.2f} \mathrm{{km/s}}$".format(0.1,0.2)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, sharex='row', sharey='row', figsize=(8,8))
plt.subplots_adjust(wspace=0.1)
plt.subplots_adjust(hspace=0.1)

# Set axis:
axes[0,0].set_xlim(0,10)
axes[0,0].set_ylim(0,30)
axes[1,0].set_xlim(0,30)
axes[1,0].set_ylim(0,90)

# Set labels:
s=14
axes[1,0].set_xlabel('$r[\mathrm{kpc}]$', fontsize=s)
axes[1,1].set_xlabel('$r[\mathrm{kpc}]$', fontsize=s)
axes[0,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=s)
axes[1,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=s)

axes[0,0].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[0,1], sat_bins[0,2] - sat_bins[0,1], 
                   sat_bins[0,1] - sat_bins[0,0]),
               transform=axes[0,0].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)

# axes[0,0].text(0.9, 0.1,
#                "$\log_{{10}}\\frac{{v_\mathrm{{max}}}}{{\mathrm{{km/s}}}} = {:.3f}$".format(
#                dex_bins[0,1]), 
#                transform=axes[0,0].transAxes, 
#                horizontalalignment='right',
#                verticalalignment='bottom',
#                fontsize=s)
# axes[0,1].text(0.9, 0.1, 
#                "$\log_{{10}}\\frac{{v_\mathrm{{max}}}}{{\mathrm{{km/s}}}} = {:.3f}$".format(
#                dex_bins[1,1]),
#                transform=axes[0,1].transAxes, 
#                horizontalalignment='right',
#                verticalalignment='bottom',
#                fontsize=s)
# axes[1,0].text(0.9, 0.1, 
#                "$\log_{{10}}\\frac{{v_\mathrm{{max}}}}{{\mathrm{{km/s}}}} = {:.3f}$".format(
#                dex_bins[2,1]),
#                transform=axes[1,0].transAxes, 
#                horizontalalignment='right',
#                verticalalignment='bottom',
#                fontsize=s)
# axes[1,1].text(0.9, 0.1,
#                "$\log_{{10}}\\frac{{v_\mathrm{{max}}}}{{\mathrm{{km/s}}}} = {:.3f}$".format(
#                dex_bins[3,1]),
#                transform=axes[1,1].transAxes, 
#                horizontalalignment='right',
#                verticalalignment='bottom',
#                fontsize=s)

fig.suptitle('Satellite Galaxies', y=1.01, fontsize=s)

# Add scatter plots:
for i, (name, entry) in enumerate(data.items()):
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['Low'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    vmax = entry['Vmax'][mask]
    rmax = entry['Rmax'][mask]
    for v, r, vm, rm in zip(v_circ, radius, vmax, rmax):
        axes[0,0].plot(r, v, c=fcolor[i], alpha=a)
        axes[0,0].plot(rm, vm, 'o', c=fcolor[i])
    
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiLow'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    vmax = entry['Vmax'][mask]
    rmax = entry['Rmax'][mask]
    for v, r, vm, rm in zip(v_circ, radius, vmax, rmax):
        axes[0,1].plot(r, v, c=fcolor[i], alpha=a)
        axes[0,1].plot(rm, vm, 'o', c=fcolor[i])
#     for v, r in zip(v_circ, radius):
#         axes[0,1].plot(r, v, c=fcolor[i], alpha=a)
        
        
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiHigh'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    vmax = entry['Vmax'][mask]
    rmax = entry['Rmax'][mask]
    for v, r, vm, rm in zip(v_circ, radius, vmax, rmax):
        axes[1,0].plot(r, v, c=fcolor[i], alpha=a)
        axes[1,0].plot(rm, vm, 'o', c=fcolor[i])
#     for v, r in zip(v_circ, radius):
#         axes[1,0].plot(r, v, c=fcolor[i], alpha=a)
        
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['High'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,1].plot(r, v, c=fcolor[i], alpha=a)
        
# axes[0].legend(loc='lower right')
plt.tight_layout()

# plt.savefig(filename, dpi=300, bbox_inches='tight')